# Surrogate model of a dynamical system by neural network

This experiment  is part of my search for fast learning models(with less data) proving my inution about finite horizon guided surrogate on small data for dynamical system. 

State variables: 𝜃
(angular position) , 𝜃˙

(angular velocity)

Input: torque(u)

Output: Cos(theta), sin(theta), thta_dot

Dynamics :

𝐷=𝑚∗𝐿∗𝐿

𝑑𝜃/𝑑𝑡=𝜃˙

𝑑𝜃˙/𝑑𝑡=(1/𝐷)∗(−𝑚∗𝑔∗𝐿∗𝑆𝑖𝑛(𝜃)−𝑏∗𝜃˙)+(1/𝐷)∗𝑢

parameters: mass of ball(m) length of string(L) drag(b)

Data collection process: random sample from action space(input space) of dynamical system and observe output of those inputs and collect multiple trajectories. In this case, we collected 20 trajectory of 1000 steps in each trajectory.

Dynamics system simultor: OpenAI gym (easy to collect) 

In [8]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import gym
import shutil

#data_collect_file='Pendulum-v1_collect.csv'
data_train_file='Pendulum-v0_train.csv'
csv_file = os.path.join(data_train_file)
openedfile=open(csv_file, 'w')
csv_writer = csv.writer(openedfile)
csv_writer.writerow(['action','cos_theta_old','sin_theta_old','theta_dot_old','cos_theta','sin_theta','theta_dot'])
env = gym.make('Pendulum-v0')
obs_old=np.zeros(3,)
state_size=4
action_size=3
prediction_model=None


HIDDEN1_UNITS=256
HIDDEN2_UNITS=128
HIDDEN3_UNITS=64



class cnn():
    def __init__(self, data_file,state_size=4,action_size=3,Testing=False):
     self.testing=Testing
     self.data= pd.read_csv(data_file)
     self.data['action']=(self.data['action']+2)/4
     self.data['cos_theta_old']=(self.data['cos_theta_old']+1)/2
     self.data['cos_theta']=(self.data['cos_theta']+1)/2
     self.data['sin_theta']=(self.data['sin_theta']+1)/2
     self.data['sin_theta_old']=(self.data['sin_theta_old']+1)/2
     self.data['theta_dot']=(self.data['theta_dot']+8)/16
     self.data['theta_dot_old']=(self.data['theta_dot_old']+8)/16
     #print('data is:',self.data)
     self.model = Net(state_size, action_size).float().cuda()
     self.model.apply(self.init_weights)
     self.optim = Adam(self.model.parameters(), lr=0.01)
     self.criterion= nn.MSELoss()
     
     
    def init_weights(self,m):
      if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
    
    def train(self,epoch) :
     batch_size=16
     data_len=self.data.shape[0]
     print('Size of data:',data_len)
     loss_track=[]
     running_loss = 0.0
     x=self.data[['action',"cos_theta_old","sin_theta_old","theta_dot_old"]].values
     y=self.data[["cos_theta","sin_theta","theta_dot"]].values
     for epo in range(epoch):
       for i in range(int(data_len/10)):
        rand_ids = np.random.randint(0, data_len, batch_size)
        #print("ids are ",rand_ids)
        self.optim.zero_grad()
        inputs, labels = x[rand_ids],y[rand_ids]
        #print("Inputs are:",inputs)
        #print("labels are:",labels)
        inputs=torch.from_numpy(inputs).cuda()
        labels=torch.from_numpy(labels).float().cuda()
        #print("type of imput",type(inputs))
        
        # forward + backward + optimize
        outputs = self.model(inputs.float())
        loss = self.criterion(outputs, labels)
        #print('loss is:',loss)
        loss.backward()
        self.optim.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.8f' %(epo + 1, i + 1, running_loss))
            loss_track.append(running_loss)
            running_loss = 0.0
     #return self.model
     #plt.figure()       
     #plt.plot(loss_track,label='loss')
     #plt.grid()
     #plt.title('Displacement Value on stream of image data')
     #plt.legend(loc="upper left")
     #plt.show()
     
def predict(model,data):
 #print('Data is:',data)
 data[0]=(data[0]+2)/4
 data[1]=(data[1]+1)/2
 data[2]=(data[2]+1)/2
 data[3]=(data[3]+8)/16
 test_data=torch.from_numpy(data).float().cuda()
 #print("Type of test data:",type(test_data))
 # input_=torch.from_numpy(np.concatenate((action,obs_old),axis=0)).cuda()
 out=model(test_data)     
 return out.detach().cpu().numpy()
         
        
class Net(nn.Module):
    def __init__(self, state_size, action_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(state_size, HIDDEN1_UNITS)
        self.fc2 = nn.Linear(HIDDEN1_UNITS, HIDDEN2_UNITS)
        self.fc3 = nn.Linear(HIDDEN2_UNITS, HIDDEN3_UNITS)
        self.fc4 = nn.Linear(HIDDEN3_UNITS, action_size)
       
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x



In [9]:
def datacollection():
  data_train=[]
  global obs_old
  env.reset() 
  for _ in range(1000):
    #env.render()
    #select a random action in range of permissible action (+2 to -2)
    action=env.action_space.sample()
    obs,reward,done,_=env.step(action) # take a random action and observe the state of system 
    # obs gives three values=> [cos(theta),sin(theta),theta_dot] range +-(1,1,8)
    csv_writer.writerow([round(action[0],3),round(obs_old[0],3),round(obs_old[1],3),round(obs_old[2],3),round(obs[0],3),round(obs[1],3),round(obs[2],3)])
    data_train.append(obs_old[0])
    obs_old=obs
  return 1    

In [10]:
for i in range(10):
 #print('In iteration:',i)
 datacollection()
openedfile.close()

self.state is: [ 2.44609283 -0.98782778] self.alt_state is : [ 2.44609283 -0.98782778]
original theta is: 2.446092831700101
original theta is: 2.4143168050866084
original theta is: 2.3934119522741657
original theta is: 2.392018525020886
original theta is: 2.4239897273710644
original theta is: 2.4745257574470836
original theta is: 2.542833695304231
original theta is: 2.6280020369145003
original theta is: 2.719142793476397
original theta is: 2.828477213751789
original theta is: 2.9364780760863805
original theta is: 3.044495029507922
original theta is: 3.1596863620553135
original theta is: 3.2644434082497917
original theta is: 3.3717080344096657
original theta is: 3.483112138500814
original theta is: 3.5836767324489354
original theta is: 3.6703369896789115
original theta is: 3.7264353504609424
original theta is: 3.7678584160216917
original theta is: 3.802137775836271
original theta is: 3.81963222186587
original theta is: 3.7992643316219565
original theta is: 3.7622178163436475
original th

original theta is: 2.7327050132956807
original theta is: 2.7707091067998215
original theta is: 2.832629880864756
original theta is: 2.909969771056748
original theta is: 3.0000626673812265
original theta is: 3.1046066281508584
original theta is: 3.2012685651802655
original theta is: 3.2945338089967993
original theta is: 3.3794552237596855
original theta is: 3.4548191525304244
original theta is: 3.5192017945826186
original theta is: 3.5640474690738704
original theta is: 3.5996652714754984
original theta is: 3.6071716400021745
original theta is: 3.6073509508308526
original theta is: 3.585920407846618
original theta is: 3.5456486047332163
original theta is: 3.483143784473426
original theta is: 3.415223992886641
original theta is: 3.3263988991845608
original theta is: 3.2419990545602877
original theta is: 3.166489290528472
original theta is: 3.079171179302877
original theta is: 2.9902102235333814
original theta is: 2.8971336199018665
original theta is: 2.8140078585622983
original theta is: 

original theta is: 3.447582058298249
original theta is: 3.3120557256129746
original theta is: 3.158210081542163
original theta is: 3.013684468844603
original theta is: 2.8736363864580996
original theta is: 2.728711540442495
original theta is: 2.6031743473750746
original theta is: 2.4890278935892387
original theta is: 2.3956328986519893
original theta is: 2.318194740346298
original theta is: 2.2643489097984713
original theta is: 2.251512276409796
original theta is: 2.278951236001881
original theta is: 2.322462868116141
original theta is: 2.3978055455122016
original theta is: 2.489885902023777
original theta is: 2.5956953173590964
original theta is: 2.7063821119765787
original theta is: 2.818525182721664
original theta is: 2.932912391856834
original theta is: 3.0478309296617487
original theta is: 3.166351888204566
original theta is: 3.2901915738205747
original theta is: 3.413045380643774
original theta is: 3.51948327187885
original theta is: 3.6241217772951764
original theta is: 3.719608

original theta is: 1.8812195735730621
original theta is: 1.8080611641428992
original theta is: 1.772161525637811
original theta is: 1.7674820458741725
original theta is: 1.8014841939207458
original theta is: 1.8711629083474979
original theta is: 1.9719702852416796
original theta is: 2.112175148683009
original theta is: 2.2931223833617587
original theta is: 2.5052244549440412
original theta is: 2.7541574540341065
original theta is: 3.0081045487135643
original theta is: 3.267878951351167
original theta is: 3.5319623305661425
original theta is: 3.7789234428543224
original theta is: 4.009595917610792
original theta is: 4.201198629885861
original theta is: 4.370888091762089
original theta is: 4.492992596610626
original theta is: 4.583296433227598
original theta is: 4.629941341187247
original theta is: 4.64877570734973
original theta is: 4.632822550814094
original theta is: 4.5879021357838825
original theta is: 4.512965780942995
original theta is: 4.405514873300371
original theta is: 4.24733

original theta is: 20.346975076479552
original theta is: 20.212219082545154
original theta is: 20.11178166827826
original theta is: 20.0333217156654
original theta is: 19.99414536676336
original theta is: 19.98985955345042
original theta is: 20.011360648243603
original theta is: 20.064210100120835
original theta is: 20.148764182921543
original theta is: 20.255563110546063
original theta is: 20.401705985830958
original theta is: 20.58905577189121
original theta is: 20.810531321695137
original theta is: 21.063682908383225
original theta is: 21.36034457981568
original theta is: 21.671619301889347
original theta is: 21.980018882709004
original theta is: 22.299125177172662
original theta is: 22.60623159119942
original theta is: 22.898138239433827
original theta is: 23.15058023585475
original theta is: 23.380374582170717
original theta is: 23.561646231497534
original theta is: 23.693532751495454
original theta is: 23.774902848914287
original theta is: 23.81126320390673
original theta is: 23.

...

original theta is: 3.4190951665030185
original theta is: 3.4317205270067195
original theta is: 3.4280938949605493
original theta is: 3.4267363227168173
original theta is: 3.4044736232917856
original theta is: 3.3745826780506527
original theta is: 3.3390136794639815
original theta is: 3.2965687739486773
original theta is: 3.2568646541209523
original theta is: 3.2130131059362825
original theta is: 3.1787096846456295
original theta is: 3.130521344651412
original theta is: 3.093841614191002
original theta is: 3.0695585492875805
original theta is: 3.052771141971672
original theta is: 3.0397545373850376
original theta is: 3.024494622936233
original theta is: 3.028273816487172
original theta is: 3.0300015916243206
original theta is: 3.0376199000441026
original theta is: 3.0454043497796497
original theta is: 3.0427344674613446
original theta is: 3.049658786332214
original theta is: 3.057028075831701
original theta is: 3.0560514588095065
original theta is: 3.067096501506097
original theta is: 3

original theta is: 3.8686932845770854
original theta is: 3.884628093864621
original theta is: 3.8683893599003314
original theta is: 3.836871619975945
original theta is: 3.7863308344595024
original theta is: 3.7076293107573646
original theta is: 3.610090700226868
original theta is: 3.497486798912182
original theta is: 3.37134083646656
original theta is: 3.2217856299798875
original theta is: 3.0624738217965906
original theta is: 2.8998451052011958
original theta is: 2.7356839282849745
original theta is: 2.5969084766694093
original theta is: 2.4833606591652506
original theta is: 2.4061153588909687
original theta is: 2.3592539203128466
original theta is: 2.3309062802686316
original theta is: 2.3384665314252016
original theta is: 2.373069863809572
original theta is: 2.4320069115347835
original theta is: 2.517952520708486
original theta is: 2.6277063726571974
original theta is: 2.750940780282904
original theta is: 2.8924527847030896
original theta is: 3.0563922499809317
original theta is: 3.

original theta is: 4.0291748693706335
original theta is: 4.154087107323078
original theta is: 4.252506529556962
original theta is: 4.327657263486647
original theta is: 4.372028013399951
original theta is: 4.387945298157024
original theta is: 4.355854358711262
original theta is: 4.297890055376002
original theta is: 4.198201133992206
original theta is: 4.074642017555762
original theta is: 3.911565157057847
original theta is: 3.7286467303040123
original theta is: 3.536873007755683
original theta is: 3.3229506518605767
original theta is: 3.0946847782861084
original theta is: 2.8600187663814154
original theta is: 2.629085469322468
original theta is: 2.4165673634908256
original theta is: 2.2380202594816674
original theta is: 2.0764864665065828
original theta is: 1.9440302707359052
original theta is: 1.8545555997709882
original theta is: 1.8106714354438198
original theta is: 1.8037969643732252
original theta is: 1.8384972139860198
original theta is: 1.9061728975452596
original theta is: 2.007

original theta is: 3.301083127659897
original theta is: 3.4277174156717574
original theta is: 3.532171946298437
original theta is: 3.630050612356891
original theta is: 3.7024627672502604
original theta is: 3.7498341654436014
original theta is: 3.7630394328220413
original theta is: 3.747432681428214
original theta is: 3.7104751742387734
original theta is: 3.6615046016715436
original theta is: 3.581570170663974
original theta is: 3.5002210658808517
original theta is: 3.395533415315376
original theta is: 3.2731196861429135
original theta is: 3.1512040770647753
original theta is: 3.0418276564499327
original theta is: 2.946313469039162
original theta is: 2.8507577826063493
original theta is: 2.753439793015145
original theta is: 2.6668171107452396
original theta is: 2.593289918033277
original theta is: 2.524448369161183
original theta is: 2.468299362813577
original theta is: 2.429758327685542
original theta is: 2.409858006528387
original theta is: 2.4126215925733168
original theta is: 2.4517

original theta is: 3.7756935939254332
original theta is: 3.6081757406885364
original theta is: 3.409447942788882
original theta is: 3.1986955066288023
original theta is: 2.9797439532721683


In [11]:
def trainnn(epo):
 neuralnet=cnn(data_train_file) 
 neuralnet.train(epo)
 torch.save(neuralnet.model.state_dict(), "./sm_train.pt")
 return 1

#This function run synchronously
def copyfile(source,dest):
    shutil.copyfile(source, dest)  

def loaddata():
 global prediction_model
 device = torch.device('cuda')
 prediction_model=Net(state_size, action_size).float()
 prediction_model.load_state_dict(torch.load("./sm_train.pt"))
 prediction_model.to(device)

def calculatetheta(cos,sin):
    if cos==-1:
      theta= np.pi
    else:
      # theta=2*arctan(sin/(1+cos))
      theta= 2*np.arctan((sin/(cos+1)))
    return theta

In [12]:
def test(sim_steps,d_step): 
 env = gym.make('Pendulum-v0')
 init_obs=env.reset() 
 pend_gym= []
 pend_sm=[]
 obs_old=np.zeros(3,)
 correction_done=0
 flag=0; flag_count=0
 angle=calculatetheta(init_obs[0],init_obs[1])
 for k in range(sim_steps):
    #env.render()
    #select a random action in range of permissible action (+2 to -2)
    action=env.action_space.sample()
    action=action.astype('double')
    obs,reward,done,_=env.step(action) # take a random action and observe the state of system 
    
    if flag==0:
      flag =1
      env.set_state([angle,init_obs[2]])
    else:
      env.set_state([angle,obs_a[2]]) 
    obs_a,reward,done,_=env.alternate_step(action)
    angle=calculatetheta(obs_a[0],obs_a[1])
    """
    #first time correction of state
    if flag==0:
      flag =1
      flag_count+=1
      obs_old=obs[0:3]   

    #correction at finite horizon   
    if d_step!=0: 
     if k%d_step==0:
      obs_old=obs[0:3]
      correction_done+=1
   
    y_predict = predict(prediction_model,np.concatenate((action,obs_old),axis=0))
    pend_gym.append(obs[0:3])
    pend_sm.append(y_predict)
    y_predict[0]=(y_predict[0]*2)-1
    y_predict[1]=(y_predict[1]*2)-1
    y_predict[2]=(y_predict[2]*16)-8
    angle=calculatetheta(y_predict[0],y_predict[1])
    #print("prediction is:",y_predict)
    #print("Theta is:",angle,"original theta is:",obs[3])
    env.set_state([angle,y_predict[2]])
    obs_a,reward,done,_=env.alternate_step(action)
    obs_old=y_predict
    """
    print("obs is:",obs,"obs alt is:",obs_a)
    print("----------------------------------------")
 """  
 df_gym=pd.DataFrame(pend_gym,columns=['cos','sin','theta_dot'])
 df_sm=pd.DataFrame(pend_sm,columns=['cos','sin','theta_dot'])
 df_gym.to_csv('gym_data.csv')
 df_sm.to_csv('sm_data.csv')
 plt.figure(figsize=(10,4))
 plt.plot(df_gym["theta_dot"],label='from gym')
 plt.plot(df_sm["theta_dot"],label='from sm')
 plt.grid()
 plt.title('Displacement Value on stream of image data')
 plt.xlabel('image_frame_number')
 plt.ylabel('Disp Value')
 plt.legend(loc="upper left")
 plt.show()

 plt.figure(figsize=(10,4))
 plt.plot(df_gym["cos"],label='from gym')
 plt.plot(df_sm["cos"],label='from sm')
 plt.grid()
 plt.title('Displacement Value on stream of image data')
 plt.xlabel('image_frame_number')
 plt.ylabel('Disp Value')
 plt.legend(loc="upper left")
 plt.show()

 plt.figure(figsize=(10,4))
 plt.plot(df_gym["sin"],label='from gym')
 plt.plot(df_sm["sin"],label='from sm')
 plt.grid()
 plt.title('Displacement Value on stream of image data')
 plt.xlabel('image_frame_number')
 plt.ylabel('Disp Value')
 plt.legend(loc="upper left")
 plt.show()
 print("Number of coorection done:",correction_done)
 print("Number of entry in flag:",flag_count)
 """

In [13]:
trainnn(1)
loaddata()

Size of data: 10000


In [15]:
test(1000,2)

self.state is: [-0.47480114 -0.169798  ] self.alt_state is : [-0.47480114 -0.169798  ]
original theta is: -0.47480114478039415
In set state: [-0.47480114478039415, -0.16979799678152352]
obs is: [ 0.88205242 -0.47115128 -0.31588816] obs alt is: [ 0.88205242 -0.47115128 -0.31588816]
----------------------------------------
original theta is: -0.4905955528670314
In set state: [-0.4905955528670313, -0.3158881617327449]
obs is: [ 0.864503   -0.50262765 -0.72080082] obs alt is: [ 0.864503   -0.50262765 -0.72080082]
----------------------------------------
original theta is: -0.5266355937861642
In set state: [-0.5266355937861641, -0.7208008183826562]
obs is: [ 0.84368257 -0.53684236 -0.80108693] obs alt is: [ 0.84368257 -0.53684236 -0.80108693]
----------------------------------------
original theta is: -0.5666899402224418
In set state: [-0.5666899402224416, -0.8010869287255513]
obs is: [ 0.80599971 -0.59191592 -1.33487821] obs alt is: [ 0.80599971 -0.59191592 -1.33487821]
-------------------

original theta is: -4.969431568756718
In set state: [1.3137537384228595, -2.7346365450618246]
obs is: [ 0.33966549  0.94054631 -1.79037271] obs alt is: [ 0.33966549  0.94054631 -1.79037271]
----------------------------------------
original theta is: -5.058950204139002
In set state: [1.2242351030405785, -1.7903727076456186]
obs is: [ 0.38246543  0.9239698  -0.91803819] obs alt is: [ 0.38246543  0.9239698  -0.91803819]
----------------------------------------
original theta is: -5.104852113448482
In set state: [1.1783331937311003, -0.9180381861895612]
obs is: [ 0.40053547  0.91628126 -0.39276059] obs alt is: [ 0.40053547  0.91628126 -0.39276059]
----------------------------------------
original theta is: -5.124490142953859
In set state: [1.1586951642257255, -0.3927605901074952]
obs is: [0.37700154 0.92621263 0.51088686] obs alt is: [0.37700154 0.92621263 0.51088686]
----------------------------------------
original theta is: -5.098945799797471
In set state: [1.1842395073821155, 0.5108868

original theta is: -13.392628343198856
In set state: [-0.8262577288397214, -4.1389998814122295]
obs is: [ 0.48698522 -0.87341021 -4.71807833] obs alt is: [ 0.48698522 -0.87341021 -4.71807833]
----------------------------------------
original theta is: -13.628532259799844
In set state: [-1.0621616454407157, -4.7180783320198865]
obs is: [ 0.24497067 -0.96953049 -5.22290487] obs alt is: [ 0.24497067 -0.96953049 -5.22290487]
----------------------------------------
original theta is: -13.889677503481465
In set state: [-1.3233068891223454, -5.222904873632594]
obs is: [-0.04467176 -0.99900172 -5.84352147] obs alt is: [-0.04467176 -0.99900172 -5.84352147]
----------------------------------------
original theta is: -14.18185357699476
In set state: [-1.6154829626356486, -5.8435214702660625]
obs is: [-0.3522602  -0.93590211 -6.30596735] obs alt is: [-0.3522602  -0.93590211 -6.30596735]
----------------------------------------
original theta is: -14.497151944489422
In set state: [-1.9307813301303

obs is: [ 0.85078815 -0.52550883 -0.27115604] obs alt is: [ 0.85078815 -0.52550883 -0.27115604]
----------------------------------------
original theta is: -19.402868989645572
In set state: [-0.5533130681068635, -0.2711560392988869]
obs is: [ 0.82514202 -0.56492534 -0.94059408] obs alt is: [ 0.82514202 -0.56492534 -0.94059408]
----------------------------------------
original theta is: -19.449898693697197
In set state: [-0.6003427721584689, -0.9405940810321067]
obs is: [ 0.77687044 -0.62966048 -1.61546657] obs alt is: [ 0.77687044 -0.62966048 -1.61546657]
----------------------------------------
original theta is: -19.530672022342415
In set state: [-0.6811161008036662, -1.615466572903948]
obs is: [ 0.71020193 -0.70399802 -1.99790441] obs alt is: [ 0.71020193 -0.70399802 -1.99790441]
----------------------------------------
original theta is: -19.630567242639895
In set state: [-0.781011321101128, -1.997904405949235]
obs is: [ 0.61453661 -0.78888831 -2.55973152] obs alt is: [ 0.61453661 

original theta is: -22.47771928522544
In set state: [2.6550219434893108, -7.6290250056393445]
obs is: [-0.65675541  0.75410366 -7.35435088] obs alt is: [-0.65675541  0.75410366 -7.35435088]
----------------------------------------
original theta is: -22.845436829249927
In set state: [2.2873043994650932, -7.354350880484352]
obs is: [-0.36972027  0.92914311 -6.75600266] obs alt is: [-0.36972027  0.92914311 -6.75600266]
----------------------------------------
original theta is: -23.183236962041345
In set state: [1.949504266674026, -6.7560026558213435]
obs is: [-0.08905793  0.99602645 -5.79063721] obs alt is: [-0.08905793  0.99602645 -5.79063721]
----------------------------------------
original theta is: -23.47276882251786
In set state: [1.6599724061979044, -5.790637209522428]
obs is: [ 0.17506258  0.98455731 -5.30290827] obs alt is: [ 0.17506258  0.98455731 -5.30290827]
----------------------------------------
original theta is: -23.737914236096344
In set state: [1.3948269926198233, -5.

...

original theta is: -48.90955331516879
In set state: [1.3559291421457602, -1.501562492651577]
obs is: [ 0.24210417  0.97025026 -0.59333541] obs alt is: [ 0.24210417  0.97025026 -0.5933354 ]
----------------------------------------
original theta is: -48.93922008544307
In set state: [1.3262623719149007, -0.5933354046171906]
obs is: [ 0.24966676  0.96833182 -0.156043  ] obs alt is: [ 0.24966676  0.96833182 -0.156043  ]
----------------------------------------
original theta is: -48.947022235679974
In set state: [1.3184602217206958, -0.15604300388410308]
obs is: [0.22395673 0.97459909 0.52927332] obs alt is: [0.22395673 0.97459909 0.52927332]
----------------------------------------
original theta is: -48.920558569744514
In set state: [1.3449238876985183, 0.5292733195564525]
obs is: [0.1714578  0.98519147 1.07126502] obs alt is: [0.1714578  0.98519147 1.07126502]
----------------------------------------
original theta is: -48.86699531894002
In set state: [1.3984871385454294, 1.071265016938

original theta is: -46.62477001363991
In set state: [-2.64247286347962, 5.78908847999689]
obs is: [-0.72710783 -0.68652327  5.1515364 ] obs alt is: [-0.72710783 -0.68652327  5.1515364 ]
----------------------------------------
original theta is: -46.36719319369703
In set state: [-2.384896043518999, 5.15153639921242]
obs is: [-0.54443761 -0.83880134  4.76762437] obs alt is: [-0.54443761 -0.83880134  4.76762437]
----------------------------------------
original theta is: -46.12881197510123
In set state: [-2.1465148249033073, 4.76762437231383]
obs is: [-0.35208522 -0.93596795  4.31840909] obs alt is: [-0.35208522 -0.93596795  4.31840909]
----------------------------------------
original theta is: -45.91289152045729
In set state: [-1.9305943702382755, 4.318409093300634]
obs is: [-0.18901974 -0.98197329  3.39268499] obs alt is: [-0.18901974 -0.98197329  3.39268499]
----------------------------------------
original theta is: -45.74325727096582
In set state: [-1.7609601207252017, 3.3926849902

obs is: [-0.334238   -0.94248871  1.81055371] obs alt is: [-0.334238   -0.94248871  1.81055372]
----------------------------------------
original theta is: -45.893890089190535
In set state: [-1.9115929390250488, 1.8105537160565597]
obs is: [-0.27608105 -0.96113436  1.2216466 ] obs alt is: [-0.27608105 -0.96113436  1.2216466 ]
----------------------------------------
original theta is: -45.83280775929997
In set state: [-1.850510609070417, 1.2216465990926346]
obs is: [-0.2573553  -0.96631685  0.38859928] obs alt is: [-0.2573553  -0.96631685  0.38859929]
----------------------------------------
original theta is: -45.81337779505491
In set state: [-1.8310806447610126, 0.3885992861880916]
obs is: [-0.27813574 -0.96054178 -0.43136814] obs alt is: [-0.27813574 -0.96054178 -0.43136814]
----------------------------------------
original theta is: -45.83494620218134
In set state: [-1.8526490518234522, -0.43136814124879197]
obs is: [-0.32276889 -0.94647781 -0.93601542] obs alt is: [-0.32276889 -0.

original theta is: -47.70639488593283
In set state: [2.5590875719097546, -2.533366698086819]
obs is: [-0.76285209  0.64657304 -2.41156901] obs alt is: [-0.76285209  0.64657304 -2.41156901]
----------------------------------------
original theta is: -47.82697333630955
In set state: [2.438509121439367, -2.4115690094077524]
obs is: [-0.69268144  0.72124367 -2.05025101] obs alt is: [-0.69268144  0.72124367 -2.05025102]
----------------------------------------
original theta is: -47.92948588701472
In set state: [2.335996570631592, -2.050251016155501]
obs is: [-0.63129009  0.77554679 -1.63969387] obs alt is: [-0.63129009  0.77554679 -1.63969387]
----------------------------------------
original theta is: -48.011470580652954
In set state: [2.254011876885314, -1.6396938749255636]
obs is: [-0.5815787   0.81349014 -1.25095104] obs alt is: [-0.5815787   0.81349014 -1.25095104]
----------------------------------------
original theta is: -48.07401813266059
In set state: [2.1914643247672267, -1.2509

In set state: [-3.0452783203555915, 2.0015637039936363]
obs is: [-0.98178045 -0.19001882  1.89733955] obs alt is: [-0.98178045 -0.19001882  1.89733955]
----------------------------------------
original theta is: -46.93270849233694
In set state: [-2.9504113427230676, 1.8973395526504806]
obs is: [-0.96446574 -0.26420795  1.52402582] obs alt is: [-0.96446574 -0.26420795  1.52402583]
----------------------------------------
original theta is: -46.85650720111923
In set state: [-2.874210051429887, 1.5240258258636077]
obs is: [-0.94433172 -0.32899485  1.35712733] obs alt is: [-0.94433172 -0.32899484  1.35712733]
----------------------------------------
original theta is: -46.78865083473853
In set state: [-2.8063536849531943, 1.357127329533859]
obs is: [-0.92906699 -0.36991151  0.87349611] obs alt is: [-0.92906699 -0.36991151  0.87349611]
----------------------------------------
original theta is: -46.74497602919545
In set state: [-2.7626788792974053, 0.873496113115781]
obs is: [-0.91591672 -0

In [144]:
LSTMnet=rnn(data_train_file)
LSTMnet.train(1)

NameError: name 'rnn' is not defined